# Setup

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Stateless apps in prod

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.application_metadata[".onEntry.show"] not in [None, "select-instance", "new-instance"]
    ).select({"On entry": lambda app: app.application_metadata[".onEntry.show"]}).order_by(
        lambda app: app["On entry"]
    ).table()

## Stateless anonymous apps in prod

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.application_metadata[".onEntry.show"] not in [None, "select-instance", "new-instance"]
        and app.application_metadata[".dataTypes.[].appLogic.allowAnonymousOnStateless", :].some(
            lambda value: value == True
        )
    ).select(
        {
            "On entry": lambda app: app.application_metadata[".onEntry.show"],
            "Anonymous dataTypes": lambda app: app.application_metadata[".dataTypes.[]", :]
            .filter(lambda dataType: dataType[".appLogic.allowAnonymousOnStateless"] == True)
            .map(lambda dataType: dataType[".id"]),
        }
    ).table()

## Enabled copy instance with more than one task and data model

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.application_metadata[".copyInstanceSettings.enabled"] == True
        and app.application_metadata[".dataTypes.[]", :]
        .filter(lambda dataType: dataType[".appLogic.classRef"] != None and dataType[".taskId"] != None)
        .map(lambda dataType: dataType[".taskId"])
        .unique()
        .length
        > 1
    ).select(
        {
            "Copy enabled": lambda app: app.application_metadata[".copyInstanceSettings.enabled"],
            "Tasks": lambda app: app.application_metadata[".dataTypes.[]", :]
            .filter(lambda dataType: dataType[".appLogic.classRef"] != None and dataType[".taskId"] != None)
            .map(lambda dataType: dataType[".taskId"])
            .unique(),
        }
    ).order_by(
        lambda app: (app.org, app.app, app.env)
    ).table()

## Apps missing `ref-data-as-pdf` dataType with `enablePdfCreation`

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: not app.application_metadata[".dataTypes.[]", :].some(
            lambda dataType: dataType[".id"] == "ref-data-as-pdf"
        )
        and app.application_metadata[".dataTypes.[]", :].some(lambda dataType: dataType[".enablePdfCreation"] == True)
    ).select({"Backend version": lambda app: app.backend_version}).table()